# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [11]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5].to_numpy()

array([0, 0, 0, 1, 0])

In [13]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [14]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [17]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [18]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [43]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_firstlayer = 10

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_secondlayer = 10

# Define the number of neurons in the output layer
output_neurons = 1

In [44]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(Dense(units=hidden_nodes_firstlayer, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(Dense(units=hidden_nodes_secondlayer, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=output_neurons, activation="sigmoid"))

In [45]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 10)                120       
                                                                 
 dense_16 (Dense)            (None, 10)                110       
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 241 (964.00 Byte)
Trainable params: 241 (964.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [46]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 2s 4ms/step - loss: 0.6662 - accuracy: 0.6255
Epoch 2/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6347 - accuracy: 0.6722
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6103 - accuracy: 0.6922
Epoch 4/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5904 - accuracy: 0.7114
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5726 - accuracy: 0.7231
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5576 - accuracy: 0.7264
Epoch 7/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5450 - accuracy: 0.7248
Epoch 8/50
38/38 [==============================] - 0s 4ms/step - loss: 0.5355 - accuracy: 0.7248
Epoch 9/50
38/38 [==============================] - 0s 8ms/step - loss: 0.5283 - accuracy: 0.7264
Epoch 10/50
38/38 [==============================] - 0s 5ms/step - loss: 0.5217 - accuracy: 0.7323
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [48]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

13/13 - 0s - loss: 0.4977 - accuracy: 0.7350 - 162ms/epoch - 12ms/step
Loss: 0.4977115988731384, Accuracy: 0.7350000143051147


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [54]:
# Set the model's file path
model_path = Path("/content/student_loans_model.keras")

# Export your model to a keras file
nn_model.save(model_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

In [59]:
from google.colab import drive
#I didn't grant colab access to my entire drive as I was able to load the file below.

### Step 1: Reload your saved model.

In [60]:
# Set the model's file path
file_path = Path('/content/student_loans_model.keras')

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [61]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test_scaled)

# Display a sample of the predictions
predictions[:5]

13/13 [==============================] - 0s 2ms/step


array([[0.41700968],
       [0.15939027],
       [0.65701586],
       [0.32757178],
       [0.7752355 ]], dtype=float32)

In [63]:
# Save the predictions to a DataFrame and round the predictions to binary results
df_predictions = pd.DataFrame(predictions.round())

# Review the DataFrame
df_predictions

,0
0,0.0
1,0.0
2,1.0
3,0.0
4,1.0
...,...
395,1.0
396,1.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [65]:
# Print the classification report with the y test data and predictions
report = classification_report(y_test, df_predictions)
print(report)

              precision    recall  f1-score   support

           0       0.69      0.74      0.71       178
           1       0.78      0.73      0.75       222

    accuracy                           0.73       400
   macro avg       0.73      0.74      0.73       400
weighted avg       0.74      0.73      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

1. The data I would need in order to build a recommendation system to recommend student loan options for students would be things like student payment histtory, where they are located, the time needed to repay the loan, how likely the individual is to pay back the loan based on credit history, and how much financial aid they qualify for to name a few.  This data is relative and appropriate because it gives an indication of previous history of the borrower and gives an indication of how big of a loan they will need and over what time frame.  It also gives an indication into history of being able to pay back loans.  

2. Based on what I chose, it would be using content-based filtering because we are using specific data points associated with the user profile and selecting features that are relevant to our model.

3.
Two real world challenges are datta privacy and diversity.  Often times, data privacy will be an issue to ensure that we are not asking for too much private information from the user that could potentially be exposed.  Also, diversity is a big issue, because we wouldn't want to stop student loans from happening based on a persons race or ethnic background.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

